In [1]:
%cd drive/My Drive/CS5814/HW3


/content/drive/My Drive/CS5814/HW3


In [2]:
pip install transformers

     |████████████████████████████████| 4.0 MB 8.8 MB/s 
     |████████████████████████████████| 77 kB 9.1 MB/s 
     |████████████████████████████████| 6.5 MB 62.9 MB/s 
     |████████████████████████████████| 895 kB 64.0 MB/s 
     |████████████████████████████████| 596 kB 78.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import json
import os
import random
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


cuda:0


In [4]:
def preprocess_data(file_loc='code_dataset.jsonl', generate_histogram=False):
    """
    Loads and processing the jsonl file,

    :param file_loc: location of target jsonl file
    :param generate_histogram: Flag to display histogram of function lengths
    :return: dataframe of preprocessed jsons
    """

    with open(file_loc, 'r') as json_file:
        json_list = list(json_file)

    code_list = []
    for json_str in json_list:
        result = json.loads(json_str)
        code_list.append(result)

    code_df = pd.DataFrame(code_list)

    total = code_df['target'].sum()
    proportion = total / code_df.shape[0]

    print("Insecure code counts: {}, Total code counts: {}, Proportion {}".format(total, code_df.shape[0], proportion))

    if generate_histogram:
        plt.hist(code_df['func'].str.len(), bins=100)
        plt.show()

    return code_df

In [5]:


def split_data(input_data, attention_data, label_data, train_ratio=0.8, val_ratio=0.10, max_len=512):
    """
    Splits data in accordance with provdied ratios, additionally discards functions with > max_len tokens
        as these will not be processed by the model will (can truncate, yet may truncate the error in the code)

    :param input_data: input functions
    :param attention_data: attention map
    :param label_data: target labels
    :param train_ratio: ratio of data to train on
    :param val_ratio: ratio of data to validate with (test is inferred from this and train)
    :param max_len: max number of tokens allowed for training date

    :return: 3 tuples for train val and test containing (input, attention, target)
    """
    # Removing excessively long elements from dataset
    valid_token_index = [i for i in range(len(input_data)) if len(input_data[i]) <= max_len]
    X_data = np.array(input_data)[valid_token_index]
    A_data = np.array(attention_data)[valid_token_index]
    Y_data = np.array(label_data)[valid_token_index]

    dataset_size = len(X_data)

    # Determining index to split dataset
    random_id = random.sample(range(dataset_size), dataset_size)
    train_split_id = int(train_ratio * dataset_size)
    val_split_id = int((train_ratio + val_ratio) * dataset_size)

    train_ids = random_id[:train_split_id]
    val_ids = random_id[train_split_id:val_split_id]
    test_ids = random_id[val_split_id:]

    X_train = torch.tensor(list(X_data[train_ids]))
    A_train = torch.tensor(list(A_data[train_ids]))
    Y_train = torch.tensor(list(Y_data[train_ids]))

    X_val = torch.tensor(list(X_data[val_ids]))
    A_val = torch.tensor(list(A_data[val_ids]))
    Y_val = torch.tensor(list(Y_data[val_ids]))

    X_test = torch.tensor(list(X_data[test_ids]))
    A_test = torch.tensor(list(A_data[test_ids]))
    Y_test = torch.tensor(list(Y_data[test_ids]))

    return (X_train, A_train, Y_train), (X_val, A_val, Y_val), (X_test, A_test, Y_test)



In [6]:

def tokenize(code_df, model_name='codebert-base'):
    """
    Apply the tokenizer from the huggingface pretrained model

    :param code_df: dataframe of preprocess code (from jsonl)
    :param model_name: model name (targeting local install)
    :return: 3 tuples for train val and test containing (input, attention, target)
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    inputs = tokenizer(code_df['func'].tolist(), truncation=False, padding='max_length')

    input_data = inputs['input_ids']
    attention_data = inputs['attention_mask']
    label_data = torch.tensor(code_df['target'].tolist())  # TODO - this can be directly converted to a np array

    return split_data(input_data, attention_data, label_data, max_len=512)



In [7]:

def train(model, train_data, val_data, epochs=5, batch_size=16, learning_rate=2e-5, validate_per=500,
          run_name="temp", run_descrption=None):
    """
    Main fine-tuning training loop for the provided model

    :param model: model loaded with predefined weights
    :param train_data: tuple of X_train, A_train, Y_train (X = inputs, A = attention, Y = target)
    :param val_data: tuple X_val, A_val, Y_val
    :param epochs: number of epochs for training
    :param batch_size: batch size (see note below about batch_hack)
    :param learning_rate: optimizer learning rate
    :param validate_per: number of weight updates before validation occurs
                            (notes: - if batch_size = 32, and validate_per = 32, validation will occur every batch
                                    - this is wrt the start of each epoch
                                    - validation will always occour at the start of each epoch (step 0))
    :param run_name: name used to saving checkpoints and log files within codebert_finetune_runs
    :param run_descrption: string that is saved to info.txt describing the run


    :return: None (models are saved in checkpoints along with log data)
    """

    # Creating dir to save logs and checkpoints, re
    dir_name = "codebert_finetune_runs/{}".format(run_name)
    if os.path.exists(dir_name):
        print("run name already exists, exiting to prevent overwriting")
        return 0
    else:
        os.makedirs(dir_name)

    # Saving run description.txt
    if run_descrption is not None:
        with open("{}/info.txt".format(dir_name), "a+") as f:
            f.write(run_descrption)

    # Unpacking data
    X_train, A_train, Y_train = train_data
    X_val, A_val, Y_val = val_data

    optimizer = AdamW(model.parameters(), lr=learning_rate)

    batch_hack = batch_size  # See note below regarding limited GPU memory

    # Initializing arrays for tracking loss
    train_loss_hist = []
    val_loss_hist = []

    # Counter to track batches (see note below related to GPU memory)
    batch_count = 0
    # validate_per_batch = int(validate_per/batch_hack)

    # Moving model to GPU if configured
    model = model.to(device)

    validate_per = int(validate_per/batch_size)

    for epoch in range(epochs):

        # Generating random index for manual shuffling of data each epoch as note using DataLoaders
        permutation = torch.randperm(X_train.shape[0])

        # Note here that only a single element is loaded at each iteration (batch size = 1) due to GPU memory constraint
        for batch_id, i in enumerate(range(0, X_train.shape[0], batch_hack)):

            # Loading batch and moving to device
            indices = permutation[i:i + batch_hack]
            batch_X, batch_Y = X_train[indices].to(device), Y_train[indices].to(device), \

            model.train()

            # Forward pass
            outputs = model(batch_X, labels=batch_Y)

            # Backward pass
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Tracking loss
            train_loss_hist.append(float(loss.item()))

            # Training output
            train_output = "Epoch:{} Step:{} Training_loss:{:.6f}".format(epoch, i, loss.item())
            print(train_output+" Training_loss_avg:{:.6f}".format(np.average(train_loss_hist[-50:])))
            with open("{}/train_loss.txt".format(dir_name), "a+") as f:
                f.write(train_output+"\n")

            # Validation
            if batch_id % validate_per == 0:
                val_loss_total = 0
                model.eval()
                print("Validating:")
                for val_badtch_id, j in tqdm(enumerate(range(0, X_val.shape[0], batch_hack))):
                    # Loading singular validation data (overwrites train data as can only load 1 intp GPU)
                    batch_X, batch_Y = X_val[j:j+batch_hack].to(device).reshape(batch_hack, -1), Y_val[j:j+batch_hack].to(device).reshape(batch_hack, -1)
                    with torch.no_grad():
                        val_outputs = model(batch_X, labels=batch_Y)
                    val_loss_total += float(val_outputs['loss'].item())

                    del batch_X
                    del batch_Y

                # Adding average loss to tracker
                val_average = val_loss_total / (val_badtch_id+1)
                val_loss_hist.append(val_average)

                # Validation output and logging
                val_output = "Epoch:{} Step:{} Val_loss:{:.6f}".format(epoch, i, val_average)
                print(val_output)
                with open("{}/val_los.txt".format(dir_name), "a+") as f:
                    f.write(val_output+"\n")

        # End of epoch checkpoint
        model.save_pretrained("{}/epoch_{}".format(dir_name, epoch + 1))



In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_8_val_8"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=2e-5,
          validate_per=100,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=2e-5, validate per 100, batch 16")


In [ ]:
gc.collect()
torch.cuda.empty_cache()
main()

Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initial

Epoch:0 Step:0 Training_loss:0.719893 Training_loss_avg:0.719893
Validating:


52it [00:07,  6.59it/s]


Epoch:0 Step:0 Val_loss:0.685511
Epoch:0 Step:8 Training_loss:0.663729 Training_loss_avg:0.691811
Epoch:0 Step:16 Training_loss:0.693459 Training_loss_avg:0.692360
Epoch:0 Step:24 Training_loss:0.685495 Training_loss_avg:0.690644
Epoch:0 Step:32 Training_loss:0.737248 Training_loss_avg:0.699965
Epoch:0 Step:40 Training_loss:0.732875 Training_loss_avg:0.705450
Epoch:0 Step:48 Training_loss:0.617226 Training_loss_avg:0.692846
Epoch:0 Step:56 Training_loss:0.643524 Training_loss_avg:0.686681
Epoch:0 Step:64 Training_loss:0.736142 Training_loss_avg:0.692177
Epoch:0 Step:72 Training_loss:0.783754 Training_loss_avg:0.701334
Epoch:0 Step:80 Training_loss:0.658209 Training_loss_avg:0.697414
Epoch:0 Step:88 Training_loss:0.741494 Training_loss_avg:0.701087
Epoch:0 Step:96 Training_loss:0.536806 Training_loss_avg:0.688450
Validating:


...

Epoch:4 Step:3072 Val_loss:0.683647
Epoch:4 Step:3080 Training_loss:0.706132 Training_loss_avg:0.692594
Epoch:4 Step:3088 Training_loss:0.686601 Training_loss_avg:0.694254
Epoch:4 Step:3096 Training_loss:0.693783 Training_loss_avg:0.696832
Epoch:4 Step:3104 Training_loss:0.696031 Training_loss_avg:0.694662
Epoch:4 Step:3112 Training_loss:0.706606 Training_loss_avg:0.693734
Epoch:4 Step:3120 Training_loss:0.683911 Training_loss_avg:0.693509
Epoch:4 Step:3128 Training_loss:0.695375 Training_loss_avg:0.696582
Epoch:4 Step:3136 Training_loss:0.645151 Training_loss_avg:0.694592
Epoch:4 Step:3144 Training_loss:0.684842 Training_loss_avg:0.695142
Epoch:4 Step:3152 Training_loss:0.679751 Training_loss_avg:0.693806
Epoch:4 Step:3160 Training_loss:0.710617 Training_loss_avg:0.692142
Epoch:4 Step:3168 Training_loss:0.660274 Training_loss_avg:0.692378
Validating:


52it [00:07,  6.59it/s]


Epoch:4 Step:3168 Val_loss:0.681969
Epoch:4 Step:3176 Training_loss:0.696548 Training_loss_avg:0.691195
Epoch:4 Step:3184 Training_loss:0.712658 Training_loss_avg:0.692967
Epoch:4 Step:3192 Training_loss:0.699628 Training_loss_avg:0.694031
Epoch:4 Step:3200 Training_loss:0.710612 Training_loss_avg:0.693177
Epoch:4 Step:3208 Training_loss:0.678805 Training_loss_avg:0.695056
Epoch:4 Step:3216 Training_loss:0.583823 Training_loss_avg:0.692275
Epoch:4 Step:3224 Training_loss:0.684445 Training_loss_avg:0.693428
Epoch:4 Step:3232 Training_loss:0.727628 Training_loss_avg:0.694211
Epoch:4 Step:3240 Training_loss:0.653907 Training_loss_avg:0.695005
Epoch:4 Step:3248 Training_loss:0.685374 Training_loss_avg:0.693906
Epoch:4 Step:3256 Training_loss:0.716797 Training_loss_avg:0.693454
Epoch:4 Step:3264 Training_loss:0.672171 Training_loss_avg:0.692038
Validating:


52it [00:07,  6.60it/s]


Epoch:4 Step:3264 Val_loss:0.681435
Epoch:4 Step:3272 Training_loss:0.729465 Training_loss_avg:0.690758
Epoch:4 Step:3280 Training_loss:0.601688 Training_loss_avg:0.689741
Epoch:4 Step:3288 Training_loss:0.667956 Training_loss_avg:0.690114
Epoch:4 Step:3296 Training_loss:0.606901 Training_loss_avg:0.687302
Epoch:4 Step:3304 Training_loss:0.645042 Training_loss_avg:0.686371
Epoch:4 Step:3312 Training_loss:0.563096 Training_loss_avg:0.684491
Epoch:4 Step:3320 Training_loss:0.593096 Training_loss_avg:0.684472
Epoch:4 Step:3328 Training_loss:0.688382 Training_loss_avg:0.683489


In [ ]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_8_lr_5e-6"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=5,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=5e-6, validate per 259, batch 8")


In [ ]:
gc.collect()
torch.cuda.empty_cache()
main()

Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initial

Epoch:0 Step:0 Training_loss:0.663632 Training_loss_avg:0.663632
Validating:


52it [00:07,  6.59it/s]


Epoch:0 Step:0 Val_loss:0.680914
Epoch:0 Step:8 Training_loss:0.675106 Training_loss_avg:0.669369
Epoch:0 Step:16 Training_loss:0.631563 Training_loss_avg:0.656767
Epoch:0 Step:24 Training_loss:0.692088 Training_loss_avg:0.665597
Epoch:0 Step:32 Training_loss:0.855427 Training_loss_avg:0.703563
Epoch:0 Step:40 Training_loss:0.707916 Training_loss_avg:0.704289
Epoch:0 Step:48 Training_loss:0.710373 Training_loss_avg:0.705158
Epoch:0 Step:56 Training_loss:0.650410 Training_loss_avg:0.698314
Epoch:0 Step:64 Training_loss:0.708048 Training_loss_avg:0.699396
Epoch:0 Step:72 Training_loss:0.679085 Training_loss_avg:0.697365
Epoch:0 Step:80 Training_loss:0.691223 Training_loss_avg:0.696806
Epoch:0 Step:88 Training_loss:0.628523 Training_loss_avg:0.691116
Epoch:0 Step:96 Training_loss:0.610138 Training_loss_avg:0.684887
Epoch:0 Step:104 Training_loss:0.697706 Training_loss_avg:0.685803
Epoch:0 Step:112 Training_loss:0.663660 Training_loss_avg:0.684327
Epoch:0 Step:120 Training_loss:0.744097 Tr

...

Epoch:4 Step:2728 Val_loss:0.621376
Epoch:4 Step:2736 Training_loss:0.422087 Training_loss_avg:0.591484
Epoch:4 Step:2744 Training_loss:0.460664 Training_loss_avg:0.588004
Epoch:4 Step:2752 Training_loss:0.482236 Training_loss_avg:0.584042
Epoch:4 Step:2760 Training_loss:0.531222 Training_loss_avg:0.581373
Epoch:4 Step:2768 Training_loss:0.387616 Training_loss_avg:0.578274
Epoch:4 Step:2776 Training_loss:0.312744 Training_loss_avg:0.570324
Epoch:4 Step:2784 Training_loss:0.741455 Training_loss_avg:0.576067
Epoch:4 Step:2792 Training_loss:0.744930 Training_loss_avg:0.578820
Epoch:4 Step:2800 Training_loss:0.352293 Training_loss_avg:0.576632
Epoch:4 Step:2808 Training_loss:0.399892 Training_loss_avg:0.575172
Epoch:4 Step:2816 Training_loss:0.669260 Training_loss_avg:0.574858
Epoch:4 Step:2824 Training_loss:0.606003 Training_loss_avg:0.572505
Epoch:4 Step:2832 Training_loss:0.423642 Training_loss_avg:0.567720
Epoch:4 Step:2840 Training_loss:0.855914 Training_loss_avg:0.574796
Epoch:4 Step

52it [00:07,  6.59it/s]


Epoch:4 Step:2976 Val_loss:0.608617
Epoch:4 Step:2984 Training_loss:0.709286 Training_loss_avg:0.585043
Epoch:4 Step:2992 Training_loss:0.477352 Training_loss_avg:0.584024
Epoch:4 Step:3000 Training_loss:0.726199 Training_loss_avg:0.583708
Epoch:4 Step:3008 Training_loss:0.528489 Training_loss_avg:0.584934
Epoch:4 Step:3016 Training_loss:0.845472 Training_loss_avg:0.589228
Epoch:4 Step:3024 Training_loss:0.768583 Training_loss_avg:0.595344
Epoch:4 Step:3032 Training_loss:0.629802 Training_loss_avg:0.594658
Epoch:4 Step:3040 Training_loss:0.523464 Training_loss_avg:0.596940
Epoch:4 Step:3048 Training_loss:0.624184 Training_loss_avg:0.591322
Epoch:4 Step:3056 Training_loss:0.598589 Training_loss_avg:0.594393
Epoch:4 Step:3064 Training_loss:0.596492 Training_loss_avg:0.597307
Epoch:4 Step:3072 Training_loss:0.624251 Training_loss_avg:0.597592
Epoch:4 Step:3080 Training_loss:0.588648 Training_loss_avg:0.595481
Epoch:4 Step:3088 Training_loss:0.628144 Training_loss_avg:0.596875
Epoch:4 Step

52it [00:07,  6.60it/s]


Epoch:4 Step:3224 Val_loss:0.616340
Epoch:4 Step:3232 Training_loss:0.613373 Training_loss_avg:0.629591
Epoch:4 Step:3240 Training_loss:0.505437 Training_loss_avg:0.622581
Epoch:4 Step:3248 Training_loss:0.709724 Training_loss_avg:0.623712
Epoch:4 Step:3256 Training_loss:0.568526 Training_loss_avg:0.625494
Epoch:4 Step:3264 Training_loss:0.678735 Training_loss_avg:0.624455
Epoch:4 Step:3272 Training_loss:0.723248 Training_loss_avg:0.629633
Epoch:4 Step:3280 Training_loss:0.607972 Training_loss_avg:0.628613
Epoch:4 Step:3288 Training_loss:0.498253 Training_loss_avg:0.626108
Epoch:4 Step:3296 Training_loss:0.500447 Training_loss_avg:0.626766
Epoch:4 Step:3304 Training_loss:0.674307 Training_loss_avg:0.620357
Epoch:4 Step:3312 Training_loss:0.503636 Training_loss_avg:0.618956
Epoch:4 Step:3320 Training_loss:0.658010 Training_loss_avg:0.619850
Epoch:4 Step:3328 Training_loss:0.745311 Training_loss_avg:0.626624


In [8]:
def main():
    """
    Main configuration function for a given finetune run
    :return: None
    """

    run_name = "test_colab_GPU_highRAM_8_lr_5e-6_10epochs"
    model_name = 'codebert-base'
    checkpoint_location = None
    online = False

    code_df = preprocess_data(file_loc='code_dataset.jsonl')
    train_data, val_data, test_data = tokenize(code_df, model_name=model_name)

    # Loading model from checkpoint if location provided
    if online:
        model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
    elif checkpoint_location is None:
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    else:
        model = AutoModelForSequenceClassification.from_pretrained(checkpoint_location)

    train(model=model,
          train_data=train_data,
          val_data=val_data,
          epochs=10,
          batch_size=8,
          learning_rate=5e-6,
          validate_per=250,
          run_name=run_name,
          run_descrption="Colab with highRam, lr=5e-6, validate per 250, batch 8, 10epochs")


In [9]:
gc.collect()
torch.cuda.empty_cache()
main()

Insecure code counts: 3729, Total code counts: 8000, Proportion 0.466125


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
Some weights of the model checkpoint at codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initial

Epoch:0 Step:0 Training_loss:0.695493 Training_loss_avg:0.695493
Validating:


52it [00:07,  6.62it/s]


Epoch:0 Step:0 Val_loss:0.744018
Epoch:0 Step:8 Training_loss:0.669641 Training_loss_avg:0.682567
Epoch:0 Step:16 Training_loss:0.613652 Training_loss_avg:0.659596
Epoch:0 Step:24 Training_loss:0.742142 Training_loss_avg:0.680232
Epoch:0 Step:32 Training_loss:0.797346 Training_loss_avg:0.703655
Epoch:0 Step:40 Training_loss:0.672647 Training_loss_avg:0.698487
Epoch:0 Step:48 Training_loss:0.787323 Training_loss_avg:0.711178
Epoch:0 Step:56 Training_loss:0.697136 Training_loss_avg:0.709423
Epoch:0 Step:64 Training_loss:0.741864 Training_loss_avg:0.713027
Epoch:0 Step:72 Training_loss:0.724831 Training_loss_avg:0.714208
Epoch:0 Step:80 Training_loss:0.725162 Training_loss_avg:0.715203
Epoch:0 Step:88 Training_loss:0.690169 Training_loss_avg:0.713117
Epoch:0 Step:96 Training_loss:0.721194 Training_loss_avg:0.713739
Epoch:0 Step:104 Training_loss:0.672107 Training_loss_avg:0.710765
Epoch:0 Step:112 Training_loss:0.680563 Training_loss_avg:0.708751
Epoch:0 Step:120 Training_loss:0.661499 Tr

...

Epoch:9 Step:2728 Val_loss:0.868331
Epoch:9 Step:2736 Training_loss:0.783048 Training_loss_avg:0.369114
Epoch:9 Step:2744 Training_loss:0.463675 Training_loss_avg:0.361874
Epoch:9 Step:2752 Training_loss:0.533964 Training_loss_avg:0.362149
Epoch:9 Step:2760 Training_loss:0.397873 Training_loss_avg:0.365172
Epoch:9 Step:2768 Training_loss:0.403498 Training_loss_avg:0.361862
Epoch:9 Step:2776 Training_loss:0.252687 Training_loss_avg:0.359155
Epoch:9 Step:2784 Training_loss:0.462970 Training_loss_avg:0.364548
Epoch:9 Step:2792 Training_loss:0.154376 Training_loss_avg:0.360917
Epoch:9 Step:2800 Training_loss:0.269746 Training_loss_avg:0.361591
Epoch:9 Step:2808 Training_loss:0.379583 Training_loss_avg:0.361071
Epoch:9 Step:2816 Training_loss:0.503162 Training_loss_avg:0.363318
Epoch:9 Step:2824 Training_loss:0.289208 Training_loss_avg:0.366082
Epoch:9 Step:2832 Training_loss:0.153327 Training_loss_avg:0.360794
Epoch:9 Step:2840 Training_loss:0.148943 Training_loss_avg:0.357096
Epoch:9 Step

52it [00:07,  6.64it/s]


Epoch:9 Step:2976 Val_loss:1.000979
Epoch:9 Step:2984 Training_loss:0.314972 Training_loss_avg:0.342199
Epoch:9 Step:2992 Training_loss:0.172112 Training_loss_avg:0.339458
Epoch:9 Step:3000 Training_loss:0.401450 Training_loss_avg:0.334917
Epoch:9 Step:3008 Training_loss:0.222571 Training_loss_avg:0.334530
Epoch:9 Step:3016 Training_loss:0.500930 Training_loss_avg:0.335684
Epoch:9 Step:3024 Training_loss:0.521555 Training_loss_avg:0.342896
Epoch:9 Step:3032 Training_loss:0.166499 Training_loss_avg:0.337035
Epoch:9 Step:3040 Training_loss:0.379225 Training_loss_avg:0.339626
Epoch:9 Step:3048 Training_loss:0.271932 Training_loss_avg:0.340628
Epoch:9 Step:3056 Training_loss:0.535706 Training_loss_avg:0.346696
Epoch:9 Step:3064 Training_loss:0.205794 Training_loss_avg:0.343328
Epoch:9 Step:3072 Training_loss:0.622270 Training_loss_avg:0.350836
Epoch:9 Step:3080 Training_loss:0.080728 Training_loss_avg:0.347972
Epoch:9 Step:3088 Training_loss:0.376027 Training_loss_avg:0.345667
Epoch:9 Step

52it [00:07,  6.63it/s]


Epoch:9 Step:3224 Val_loss:0.921827
Epoch:9 Step:3232 Training_loss:0.487739 Training_loss_avg:0.331065
Epoch:9 Step:3240 Training_loss:0.428777 Training_loss_avg:0.336661
Epoch:9 Step:3248 Training_loss:0.204582 Training_loss_avg:0.336748
Epoch:9 Step:3256 Training_loss:0.387032 Training_loss_avg:0.338654
Epoch:9 Step:3264 Training_loss:0.163260 Training_loss_avg:0.337282
Epoch:9 Step:3272 Training_loss:0.291582 Training_loss_avg:0.338721
Epoch:9 Step:3280 Training_loss:0.446353 Training_loss_avg:0.343341
Epoch:9 Step:3288 Training_loss:0.829804 Training_loss_avg:0.352873
Epoch:9 Step:3296 Training_loss:0.667654 Training_loss_avg:0.356335
Epoch:9 Step:3304 Training_loss:0.663269 Training_loss_avg:0.362028
Epoch:9 Step:3312 Training_loss:0.566651 Training_loss_avg:0.366491
Epoch:9 Step:3320 Training_loss:0.240226 Training_loss_avg:0.367521
Epoch:9 Step:3328 Training_loss:0.755585 Training_loss_avg:0.374802
